In [1]:
import gym
import torch
import pandas as pd
import numpy as np
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"   

from elegantrl.run import *
from neo_finrl.ccxt.data_fetch_ccxt import ccxt_fetch_data
from neo_finrl.ccxt.preprocess_ccxt import preprocess_btc

from neo_finrl.ccxt.env_btc_ccxt import BitcoinEnv
from elegantrl.agent import *

In [2]:
# url = 'Data/crypto_quant/btc/total_data.csv'
# url = 'Data/crypto_quant/btc/total_block.csv'
# url = 'Data/crypto_quant/btc/total_hour.csv'
url = 'Data/crypto_quant/btc/market_data_total.csv'
url = 'Data/crypto_quant/btc/market_data_total_all_exchange.csv'

In [3]:
df = pd.read_csv(url, parse_dates=True).fillna(value=0)

In [4]:
'''data_fetch'''
# df = ccxt_fetch_data(start = '20200602 00:00:00', end = '20210609 00:00:00',
#                     pair = 'DOGE/USDT', period = '1m')
# df = ccxt_fetch_data(start = '20190418 11:40:00', end = '20210628 00:00:00',
#                     pair = 'BNB/USDT', period = '5m')
df

,time,open,high,low,close,volume,open_interest,funding_rates,taker_buy_volume,taker_sell_volume,taker_buy_ratio,taker_sell_ratio,taker_buy_sell_ratio,long_liquidations,short_liquidations,long_liquidations_usd,short_liquidations_usd
0,2020-01-01 00:00:00,7193.015439,7193.496755,7182.815371,7185.404910,115.419148,1.047269e+09,0.011658,9.836582e+05,5.686439e+06,0.147473,0.852527,0.172983,0.0,0.000000,0.0,0.0000
1,2020-01-01 00:01:00,7172.657634,7173.579323,7168.326232,7169.762129,51.064843,1.047805e+09,0.011647,1.161364e+06,1.343338e+06,0.463674,0.536326,0.864536,0.0,0.000000,0.0,0.0000
2,2020-01-01 00:02:00,7179.699960,7180.523442,7177.268917,7179.137047,34.827771,1.047696e+09,0.011644,8.060659e+05,2.252758e+06,0.263522,0.736478,0.357813,0.0,0.000000,0.0,0.0000
3,2020-01-01 00:03:00,7193.951046,7198.598589,7193.108906,7196.707575,22.708619,1.048363e+09,0.011641,2.464693e+06,8.085943e+05,0.752972,0.247028,3.048121,0.0,0.000000,0.0,0.0000
4,2020-01-01 00:04:00,7185.993896,7185.995767,7180.364448,7181.069773,30.758360,1.047958e+09,0.011643,5.883227e+05,2.312188e+06,0.202834,0.797166,0.254444,0.0,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796316,2021-07-06 23:56:00,34151.223196,34169.391045,34147.292454,34168.775640,99.670635,4.349065e+09,-0.017584,7.540093e+06,5.367953e+06,0.584139,0.415861,1.404650,0.0,0.000000,0.0,0.0000
796317,2021-07-06 23:57:00,34175.433050,34308.046523,34170.757729,34305.064154,408.103111,4.350150e+09,-0.017571,5.312143e+07,1.239381e+07,0.810826,0.189174,4.286127,0.0,2.450655,0.0,83971.3073
796318,2021-07-06 23:58:00,34307.796203,34338.065694,34215.269658,34248.138593,463.143644,4.347514e+09,-0.017557,4.572928e+07,4.609810e+07,0.497992,0.502008,0.991999,0.0,2.116023,0.0,72930.0000
796319,2021-07-06 23:59:00,34247.882044,34270.453353,34216.234589,34230.576260,118.935728,4.347795e+09,-0.017551,1.459520e+07,1.290423e+07,0.530746,0.469254,1.131040,0.0,0.000000,0.0,0.0000


In [5]:
df = df.set_index('time')

In [6]:
# from talib.abstract import *

# # df['r'] = np.log(df['close'] / df['close'].shift(1))
# # df['s'] = df['close'].rolling(30).mean()
# # df['m'] = df['r'].rolling(30).mean()
# # df['v'] = df['r'].rolling(30).std()
# # df["daily_return"] = df.close.pct_change(1)

# # Price Transform
# df['AVGPRICE'] = AVGPRICE(df['open'], df['high'], df['low'], df['close'])
# df['MEDPRICE'] = MEDPRICE(df['high'], df['low'])
# df['TYPPRICE'] = TYPPRICE(df['high'], df['low'], df['close'])
# df['WCLPRICE'] = WCLPRICE(df['high'], df['low'], df['close'])

# # Overlap Studies
# df['upperband'], middleband, df['lowerband'] = BBANDS(df['close'], timeperiod=5, matype=0)
# # df['upperband'], df['middleband'], df['lowerband'] = BBANDS(df['close'], timeperiod=5, matype=0)
# df['EMA'] = EMA(df['close'])    
# df['SMA'] = SMA(df['close'], timeperiod=30)
# # df['MIDPOINT'] = MIDPOINT(df['close'], timeperiod=30)
# # df['MIDPRICE'] = MIDPRICE(df['high'], df['low'], timeperiod=30)

# # Momentum Indicators
# # df['MACD'], df['MACDSIGNAL'], df['MACDHIST'] = MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
# df['RSI'] = RSI(df['close'], timeperiod=30)
# # df['FASTK'], df['FASTD'] = STOCHRSI(df['close'], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
# df['CCI'] = CCI(df['high'], df['low'], df['close'], timeperiod=30)
# df['DX'] = DX(df['high'], df['low'], df['close'], timeperiod=30)
# # df['slowk'], df['slowd'] = STOCH(df['high'], df['low'], df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
# # df['fastkf'], df['fastdf'] = STOCHF(df['high'], df['low'], df['close'], fastk_period=5, fastd_period=3, fastd_matype=0)

#  # Volatility Indicator Functions
# # df['ATR'] = ATR(df, timeperiod=30)
# # df['ATR%'] = df['ATR'] / df['close']
# # df['NATR'] = NATR(df, timeperiod=30)
# # df['TRANGE'] = TRANGE(df)

In [7]:
# def create_lag_data(lag=50):
#     df_lagged = df.copy()
#     for window in range(1, lag + 1):
#         shifted = df.shift(window)
#         shifted.columns = [x + "_lag_" + str(window) for x in df.columns]
#         df_lagged = pd.concat((df_lagged, shifted), axis=1)
#     df_lagged = df_lagged.dropna()
#     return df_lagged

In [8]:
# df = create_lag_data()
# df = df.dropna()

In [9]:
# df = df[350000:]
df

,open,high,low,close,volume,open_interest,funding_rates,taker_buy_volume,taker_sell_volume,taker_buy_ratio,taker_sell_ratio,taker_buy_sell_ratio,long_liquidations,short_liquidations,long_liquidations_usd,short_liquidations_usd
time,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,7193.015439,7193.496755,7182.815371,7185.404910,115.419148,1.047269e+09,0.011658,9.836582e+05,5.686439e+06,0.147473,0.852527,0.172983,0.0,0.000000,0.0,0.0000
2020-01-01 00:01:00,7172.657634,7173.579323,7168.326232,7169.762129,51.064843,1.047805e+09,0.011647,1.161364e+06,1.343338e+06,0.463674,0.536326,0.864536,0.0,0.000000,0.0,0.0000
2020-01-01 00:02:00,7179.699960,7180.523442,7177.268917,7179.137047,34.827771,1.047696e+09,0.011644,8.060659e+05,2.252758e+06,0.263522,0.736478,0.357813,0.0,0.000000,0.0,0.0000
2020-01-01 00:03:00,7193.951046,7198.598589,7193.108906,7196.707575,22.708619,1.048363e+09,0.011641,2.464693e+06,8.085943e+05,0.752972,0.247028,3.048121,0.0,0.000000,0.0,0.0000
2020-01-01 00:04:00,7185.993896,7185.995767,7180.364448,7181.069773,30.758360,1.047958e+09,0.011643,5.883227e+05,2.312188e+06,0.202834,0.797166,0.254444,0.0,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-06 23:56:00,34151.223196,34169.391045,34147.292454,34168.775640,99.670635,4.349065e+09,-0.017584,7.540093e+06,5.367953e+06,0.584139,0.415861,1.404650,0.0,0.000000,0.0,0.0000
2021-07-06 23:57:00,34175.433050,34308.046523,34170.757729,34305.064154,408.103111,4.350150e+09,-0.017571,5.312143e+07,1.239381e+07,0.810826,0.189174,4.286127,0.0,2.450655,0.0,83971.3073
2021-07-06 23:58:00,34307.796203,34338.065694,34215.269658,34248.138593,463.143644,4.347514e+09,-0.017557,4.572928e+07,4.609810e+07,0.497992,0.502008,0.991999,0.0,2.116023,0.0,72930.0000


In [10]:
data_ary = preprocess_btc(df, is_lag=False)

                            open         high          low        close  \
time                                                                      
2020-01-01 00:00:00  7193.015439  7193.496755  7182.815371  7185.404910   
2020-01-01 00:01:00  7172.657634  7173.579323  7168.326232  7169.762129   
2020-01-01 00:02:00  7179.699960  7180.523442  7177.268917  7179.137047   
2020-01-01 00:03:00  7193.951046  7198.598589  7193.108906  7196.707575   
2020-01-01 00:04:00  7185.993896  7185.995767  7180.364448  7181.069773   

                         volume  open_interest  funding_rates  \
time                                                            
2020-01-01 00:00:00  115.419148   1.047269e+09       0.011658   
2020-01-01 00:01:00   51.064843   1.047805e+09       0.011647   
2020-01-01 00:02:00   34.827771   1.047696e+09       0.011644   
2020-01-01 00:03:00   22.708619   1.048363e+09       0.011641   
2020-01-01 00:04:00   30.758360   1.047958e+09       0.011643   

                  

In [11]:
df = df.reset_index()

In [12]:
data_ary.shape

(796321, 16)

In [13]:
gpu_id = 2
args = Arguments(agent=None, env=None, gpu_id=gpu_id)
args.agent = AgentDoubleDQN()

In [14]:
'''choose environment'''
args.env = BitcoinEnv(processed_ary = data_ary, if_train=True, if_test=False)
args.env_eval = BitcoinEnv(processed_ary = data_ary, if_train=False, if_test=False)
args.net_dim = 2 ** 12 # change a default hyper-parameters
args.batch_size = 2 ** 11

In [ ]:
train_and_evaluate(args)

| GPU id: 2, cwd: ./AgentDoubleDQN/BitcoinEnv_2
| Remove history
ID      Step      MaxR |    avgR      stdR       objA      objC |  avgS  stdS


In [ ]:
args = Arguments(agent=None, env=None, gpu_id=gpu_id)
args.agent = AgentDoubleDQN()
args.env = BitcoinEnv(processed_ary = data_ary, if_train=False, if_test=True)
args.net_dim = 2 ** 12 # change a default hyper-parameters
args.batch_size = 2 ** 11
args.if_remove = False
args.cwd = './AgentDoubleDQN/BitcoinEnv_'+str(gpu_id)
args.init_before_training()

In [ ]:
prediction = args.env.trade_prediction(args, torch, df)

In [ ]:
args.env.backtest_plot(prediction, df)

In [ ]:
# Draw the graph
import matplotlib.pyplot as plt
%matplotlib inline

BitcoinEnv(processed_ary = data_ary, if_train=False, if_test=True)\
.draw_cumulative_return(self = args.env, args = args, _torch = torch)